# Credit Risk Resampling Techniques

In [1]:
import warnings


warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from collections import Counter
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import ClusterCentroids
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
import pickle






# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('forlex.csv')
df = pd.read_csv(file_path)
df.head()

,ID,LIMIT_BAL,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_pay,Total_Missed_Pymts,Payment_Ratio,Total_Bill_AMT,Total_Payment,%_Amount_Paid,Risk_Level
0,1,20000.0,1,24,1,1,0,0,0,0,...,0.0,0.0,0.0,1,2,33.333333,7704.0,689.0,8.943406,Moderate
1,2,120000.0,2,26,0,1,0,0,0,1,...,1000.0,0.0,2000.0,1,2,33.333333,17077.0,5000.0,29.279147,Moderate
2,3,90000.0,2,34,0,0,0,0,0,0,...,1000.0,1000.0,5000.0,0,0,0.000000,101653.0,11018.0,10.838834,NaN
3,4,50000.0,1,37,0,0,0,0,0,0,...,1100.0,1069.0,1000.0,0,0,0.000000,231334.0,8388.0,3.625926,NaN
4,5,50000.0,1,57,0,0,0,0,0,0,...,9000.0,689.0,679.0,0,0,0.000000,109339.0,59049.0,54.005433,NaN


In [4]:
X=df[['MARRIAGE', 'AGE','PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6','Total_Missed_Pymts', 'Payment_Ratio','Total_Payment']]


In [5]:
y=df[['default_pay']]

# Split the Data into Training and Testing

In [6]:
# Check the balance of our target values

y['default_pay'].value_counts()

0    23364
1     6636
Name: default_pay, dtype: int64

In [7]:
# Create X_train, X_test, y_train, y_test
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [8]:
print(f'''% Positive class in Train ={np.round(y_train.value_counts(normalize=True)[1] *100, 2)}''')

% Positive class in Train =default_pay
1              22.08
dtype: float64


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
# YOUR CODE HERE
scaler = StandardScaler()



In [10]:
 #Fit the Standard Scaler with the training data
 #When fitting scaling functions, only train on the training dataset
#YOUR CODE HERE
X_scaler = scaler.fit(X_train)


In [11]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


### Clustered Centroids

In [12]:
# Resample the data using the ClusterCentroids resampler
# YOUR CODE HERE


# Fit the data using `ClusterCentroids` and check the count of each class

cc = ClusterCentroids(random_state=1)
Clust_X_resampled, Clust_y_resampled = cc.fit_resample(X_train_scaled, y_train)



# View the count of target classes with Counter
# YOUR CODE HERE

#Counter(Clust_y_resampled.value_counts)

In [13]:
Counter(y_train.value_counts())

Counter({17532: 1, 4968: 1})

In [14]:
pickle.dump(cc,open('cluster.pkl', 'wb'))


In [15]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression
c_model = LogisticRegression(solver='lbfgs', random_state=1)
c_model.fit(Clust_X_resampled, Clust_y_resampled)

LogisticRegression(random_state=1)

In [16]:
# Calculate the balanced accuracy score
# YOUR CODE HERE
c_y_pred = c_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, c_y_pred)

0.6572525929873385

In [17]:
print(f"Training Data Score: {c_model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {c_model.score(X_test_scaled, y_test)}")

Training Data Score: 0.6190222222222223
Testing Data Score: 0.6237333333333334


In [18]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, c_y_pred)

array([[3481, 2351],
       [ 471, 1197]], dtype=int64)

In [19]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, c_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.88      0.60      0.72      0.71      0.65      0.42      5832
          1       0.34      0.72      0.60      0.46      0.65      0.43      1668

avg / total       0.76      0.62      0.69      0.66      0.65      0.43      7500

